In [106]:
import pandas as pd
df1 = pd.read_csv("Uniace_1.csv")
df2 = pd.read_csv("Uniace_2.csv")
df3 = pd.read_csv("Uniace_3.csv")
df = pd.concat([df1, df2, df3], ignore_index=True)

In [107]:
df.loc[(df["Name"] == 0) & (df["ma_path"] != 0), "Name"] = df["ma_path"]
df["MA Referrer"] = df["MA Referrer"].replace(0, "Uniace")
df["Date"] = df["Date"].str.replace("-", "/")
df = df.fillna(0)
df = df.drop(columns = ["ma_path","Title","MA URL","Template Id", "List Id", "Form Id", "Campaign Id", "Campaign Name","Scenario Id", "URL", "Link", "Tag"], axis=1)

In [53]:
df["Type"].unique()

array(['page', 'templates_open', 'identify', 'list_addition',
       'templates_click', 'form', 'templates_unsubscribe'], dtype=object)

In [108]:
def try_parse_date(date_str):
    # Try different date formats
    for fmt in ('%d/%m/%Y %H:%M','%d/%m/%Y %H:%M:%S'):
        try:
            return pd.to_datetime(date_str, format=fmt)
        except ValueError:
            continue
    return pd.Timestamp('2022-01-01 00:00:00')
# Apply the function to the 'Date' column and create a new 'Date2' column
df['Date'] = df['Date'].apply(lambda x: try_parse_date(x))

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 94080 entries, 0 to 94079
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   Email        94080 non-null  object        
 1   Type         94080 non-null  object        
 2   Name         94080 non-null  object        
 3   MA Referrer  94080 non-null  object        
 4   IP Address   94080 non-null  object        
 5   cuid         94080 non-null  object        
 6   Date         94080 non-null  datetime64[ns]
 7   Message Id   94080 non-null  object        
dtypes: datetime64[ns](1), object(7)
memory usage: 5.7+ MB


In [139]:
# Define classification function for email
def classify_email(email):
    if isinstance(email, str): 
        email = email.lower()
        if 'gmail.com' in email or 'yahoo.com' in email:
            return 'personal_domains'

        elif 'outlook.com' in email or 'icloud.com' in email:
            return 'officer'
        
        elif 'uef' in email:
            return 'uef'

        elif 'uel' in email:
            return 'uel'

        elif 'ueh' in email:
            return 'ueh'

        elif 'ctu' in email:
            return 'ctu'

        elif 'uniace' in email:
            return 'uniace'

        elif any(domain in email for domain in ['ute', 'ussh', 'due', 'hanu', 'phenikaa-uni', 'ueb', 'st', 
                                                'hcmut', 'hust', 'utb', 'vimaru', 'fpt', 'hoasen', 'ftu', 
                                                'ntu', 'vanlang','dau', 'nuce', 'glos' ,'med','agu', 'hcmiu', 
                                                'ptit', 'ou', 'westernsydney', 'hou', 'stu']):
            return 'VN students'

        else:
            return 'Others'
    return 'Unsigned_account'

# Apply the function to the DataFrame to create a new column 'Classify_email'
df['Classify_email'] = df['Email'].apply(classify_email)



In [126]:
e_u = df["Email"]
domain_count = {}  # Dictionary to store domain counts

for i in e_u:
    if isinstance(i, str):  # Check if the value is a string
        if '@' in i:  # Ensure the string contains an '@'
            domain = i[i.index('@'):]  # Extract the domain part
            if domain in domain_count:
                domain_count[domain] += 1  # Increment count if domain already exists
            else:
                domain_count[domain] = 1  # Add new domain to the dictionary with count 1

# Print the domain counts
for domain, count in domain_count.items():
    print(f"{domain}: {count}")


@gmail.com: 33417
@outlook.com.vn: 857
@student.hcmute.edu.vn: 16
@hcmussh.edu.vn: 4
@ou.edu.vn: 32
@student.westernsydney.edu.vn: 22
@zwoho.com: 5
@eoopy.com: 3
@students.hou.edu.vn: 16
@outlook.com: 1137
@gmai.com: 78
@yahoo.com: 9
@gmail.cm: 3
@nguyenngoc.2611: 3
@mail.tdc.edu.vn: 3
@due.udn.vn: 3
@s.hanu.edu.vn: 6
@st.phenikaa-uni.edu.vn: 24
@ldxh.edu.vn: 3
@sv.ussh.edu.vn: 23
@ueb.edu.vn: 3
@st.hcmuaf.edu.vn: 3
@st.neu.edu.vn: 13
@st.buh.edu.vn: 72
@gmail.com.vn: 3
@ms.uit.edu.vn: 3
@thanglong.edu.vn: 3
@student.hcmus.edu.vn: 3
@icloud.com: 174
@uef.edu.vn: 792
@st.uel.edu.vn: 340
@mail.com: 15
@hotmail.com: 12
@vku.udn.vn: 6
@gnail.com: 3
@gmail.con: 18
@hcmut.edu.vn: 45
@st.ueh.edu.vn: 636
@sis.hust.edu.vn: 6
@utb.edu.vn: 6
@st.vimaru.edu.vn: 7
@fpt.edu.vn: 34
@gentis.com.vn: 3
@sinhvien.hoasen.edu.vn: 3
@huutoan.com: 3
@ftu.edu.vn: 62
@gmail.co: 3
@gmial.com: 3
@cuoly.com: 3
@student.ctu.edu.vn: 342
@email.com: 3
@ntu.edu.vn: 7
@outlock.com.vn: 3
@outlook.com.com.vn: 3
@medvnu.

In [134]:
filtered_df = df[df['Email'].str.contains('ueh', na=False)]
filtered_df 

,Email,Type,IP Address,cuid,Date,Message Id,Classify_email,Classify_name,Classify_Referrer
21428,duyenle.31201023159@st.ueh.edu.vn,list_addition,0,0,2021-08-14 12:56:00,0,VN students,0,Others
21674,yenhuynh.31201025918@st.ueh.edu.vn,list_addition,0,0,2021-08-14 12:58:00,0,VN students,0,Others
27906,duyenle.31201023159@st.ueh.edu.vn,templates_open,0,0,2021-08-19 07:57:00,0,VN students,Young_talent,Others
30181,duyenle.31201023159@st.ueh.edu.vn,templates_open,0,0,2021-08-20 23:29:00,0,VN students,Young_talent,Others
32131,longnguyen.31201021148@st.ueh.edu.vn,templates_open,0,0,2022-01-01 00:00:00,0,VN students,Young_talent,Others
...,...,...,...,...,...,...,...,...,...
91537,longnguyen.31201021148@st.ueh.edu.vn,identify,45.32.48.212,cfba9965588c5eb75883b6efce642290,2021-08-13 15:21:54,0,VN students,0,Others
91538,longnguyen.31201021148@st.ueh.edu.vn,identify,45.32.48.212,167fd928cd8e6c6e7fcf083b8b6f8ceb,2021-08-13 15:21:42,0,VN students,0,Others
91539,longnguyen.31201021148@st.ueh.edu.vn,page,14.174.233.230,b830ed5c-3b97-4a99-ba6b-76e236823d22,2021-08-13 15:21:34,0,VN students,Khóa học,Others
93441,hieudiu.31191025944@st.ueh.edu.vn,identify,45.32.48.212,48357ace3fd87e2495f02222010ccfa8,2021-08-12 16:47:00,0,VN students,0,Others


In [140]:
df["Classify_email"].value_counts()

Unsigned_account    55096
personal_domains    33432
officer              2171
uef                   795
ueh                   636
uniace                612
VN students           458
ctu                   342
uel                   340
Others                198
Name: Classify_email, dtype: int64

In [111]:
# Define classification function for names
def classify_name(name):
    if name != 0:
        # Convert name to lowercase for case-insensitive comparison
        name_lower = name.lower()
        
        
        if any(keyword in name_lower for keyword in ['trial' ]):
            return 'Trial'
        
        if any(keyword in name_lower for keyword in ['remind' ]):
            return 'Remind'
        
        if any(keyword in name_lower for keyword in ['young-talent' ]):
            return 'Young_talent'
        
        if any(keyword in name_lower for keyword in [ 'register', 'registration','new account', 'newsletter']):
            return 'Registration'
        
        # Classify based on specific patterns or keywords
        if any(keyword in name_lower for keyword in ['excel' ]):
            return 'Excel'
        
        if any(keyword in name_lower for keyword in ['cách', 'hướng dẫn', 'tự động hóa', 'xây dựng', 'quy tắc',
                                                     'để', 'cam kết', 'làm gì', 'đăng ký']):
            return 'Hướng dẫn'
        
        elif any(keyword in name_lower for keyword in ['là gì', 'tại sao', 'tổng hợp','theo đuổi', 'bị kẹt', 'trả lời', 'lợi ích']):
            return 'Giải đáp'
        
        elif any(keyword in name_lower for keyword in ['trở thành', 'chương trình','yêu cầu', 'bức tranh', 'chia sẻ', 'kiến thức'
                                                      ,'điểm qua', 'học từ giai đoạn', 'tìm hiểu', 'junie vu', "văn hóa hối hả"]):
            return 'Cung cấp thông tin'
        
        elif any(keyword in name_lower for keyword in ['khóa học', 'uniace', "học"]):
            return 'Khóa học'
 
    return name 

# Apply the function to the DataFrame to create a new column 'Classify_name'
df['Classify_name'] = df['Name'].apply(classify_name)


In [112]:
# Define classification function for names
def classify_referrer(name):
    if name != 0:
        # Convert name to lowercase for case-insensitive comparison
        name_lower = name.lower()
        
        
        if any(keyword in name_lower for keyword in ['google' ]):
            return 'Google'
        
        if any(keyword in name_lower for keyword in ['coccoc' ]):
            return 'CocCoc'
        
        if any(keyword in name_lower for keyword in ['yahoo' ]):
            return 'Yahoo'
        
        if any(keyword in name_lower for keyword in ['uniace' ]):
            return 'Uniace'
        
        if any(keyword in name_lower for keyword in ['bing' ]):
            return 'Bing'
        
        if any(keyword in name_lower for keyword in ['facebook' ]):
            return 'Meta (Facebook)'
        
        if any(keyword in name_lower for keyword in ['beacons' ]):
            return 'Beacons'
        
        if any(keyword in name_lower for keyword in ['zalo' ]):
             return 'Zalo'
        
        if any(keyword in name_lower for keyword in ['youtube' ]):
            return 'Youtube'
 
    return 'Others'

# Apply the function to the DataFrame to create a new column 'Classify_name'
df['Classify_Referrer'] = df['MA Referrer'].apply(classify_referrer)


In [113]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 94080 entries, 0 to 94079
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   Email              94080 non-null  object        
 1   Type               94080 non-null  object        
 2   Name               94080 non-null  object        
 3   MA Referrer        94080 non-null  object        
 4   IP Address         94080 non-null  object        
 5   cuid               94080 non-null  object        
 6   Date               94080 non-null  datetime64[ns]
 7   Message Id         94080 non-null  object        
 8   Classify_email     94080 non-null  object        
 9   Classify_name      94080 non-null  object        
 10  Classify_Referrer  94080 non-null  object        
dtypes: datetime64[ns](1), object(10)
memory usage: 7.9+ MB


In [141]:
df = df.drop(columns=["Name", "MA Referrer"])
df.to_csv("d56_final.csv")